<a href="https://colab.research.google.com/github/joedmugo10/AI-MoodBoost-prueba/blob/main/chatbot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import requests
from bs4 import BeautifulSoup

!pip install transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import requests
from bs4 import BeautifulSoup

# Función para traducir texto usando la API de traducción de Google
def translate_text(text, target_language="en"):
    url = "https://translate.googleapis.com/translate_a/single"
    params = {
        "client": "gtx",
        "sl": "auto",
        "tl": target_language,
        "dt": "t",
        "q": text
    }

    response = requests.get(url, params=params)
    translations = response.json()
    translated_text = translations[0][0][0]
    return translated_text

# Instancia del tokenizador y el modelo preentrenado DialoGPT-Medium de Microsoft
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Inicializa el historial del chat
chat_history_ids = None

# Conversación de 5 líneas
for step in range(5):
    # Traduce la entrada del usuario al inglés
    user_input = input(">> Usuario:")
    user_input_english = translate_text(user_input, target_language="en")

    # Codifica la nueva entrada del usuario en inglés y agrega el token de fin de secuencia (eos_token)
    new_user_input_ids = tokenizer.encode(user_input_english + tokenizer.eos_token, return_tensors='pt')

    # Añade los nuevos tokens de entrada del usuario al historial del chat
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # Genera una respuesta en inglés limitando la longitud total del historial del chat a 1000 tokens
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Decodifica la respuesta en inglés
    english_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Traduce la respuesta al español
    spanish_response = translate_text(english_response, target_language="es")

    # Imprime la respuesta en español
    print("DialoGPT: {}".format(spanish_response))




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.7 MB/s eta 0:00:00


>> Usuario:hola , como estas ?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: ¿Estoy bien, cómo estás?
>> Usuario:me siento feliz 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Eso es bueno.
>> Usuario:me siento triste


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: Yo también me siento triste.


KeyboardInterrupt: ignored